In [ ]:
# Imports

import os
import shutil

import re
import yaml

import geopandas as gpd
import numpy as np
import rasterio as rio
from rasterio.mask import mask as rio_mask

from vercye_ops.apsim.convert_shapefile_to_geojson import convert_shapefile_to_geojson

In [ ]:
# Define parameters and paths
# Please refer to the documentation to understand on how to define the config.

SHAPEFILE_PATH = '/path/to/shapefile.shp'
ADMIN_COLUMN_NAME = 'NAME_2' # Name of the column that contains the administrative division level names for the level of interest in you shapefile.
SHAPEFILE_CENTROID_PROJECTION_EPSG = 4326
GEOJSONS_FOLDER = "/path/to/geosons" # Output folder where to save the extracted geojsons

# Optional: Reference raster path - used to get the projection and resolution of the raster for rasterizing the polygons
# Typically should be one of the LAI rasters for the region
# If none, no check if all regions can be rasterized with at least one pixel will be performed
REFERENCE_RASTER_PATH = None

# In the beginning you want to start out with a snakemake config file that is completely filled out except for the "regions" field
# This script will help you fill out the "regions" field with the regions extracted from the shapefile
SNAKEFILE_CONFIG = "" # Config file for the snakemake pipeline
OUTPUT_DIR = '' # Directory to save the new head_dir structure and files

# TODO allow different apsim templates for different regions/timepoints?
APSIM_TEMPLATE_PATH = ''

In [ ]:
# Extracts geojsons in with their corresponding vercye-style directories from a shapefile
# The geojsons are used as a base for the yieldstudy regions

convert_shapefile_to_geojson(shp_fpath=SHAPEFILE_PATH, admin_name_col=ADMIN_COLUMN_NAME, projection_epsg=SHAPEFILE_CENTROID_PROJECTION_EPSG, output_head_dir=GEOJSONS_FOLDER)

In [ ]:
# Prints the names of the regions that have at least one pixel after rasterization
# Copy the names of the regions that you want to keep from the output into your snakemake config under regions

for f in os.listdir(GEOJSONS_FOLDER):
    geojson_folder_path = os.path.join(GEOJSONS_FOLDER, f)
    if not os.path.isdir(geojson_folder_path):
        continue

    if REFERENCE_RASTER_PATH is None:
        print(f"- '{f}'")
        continue

    # Identify number of pixels in rasterized shapefile
    shp = gpd.read_file(os.path.join(geojson_folder_path, f + ".geojson"))
    src = rio.open(REFERENCE_RASTER_PATH)
    masked_src, masked_transform = rio_mask(src, shp.geometry, crop=False, nodata=0, indexes=1)
    num_pixels_rasterized = np.count_nonzero(masked_src)

    # Prints region name if at least one pixel is rasterized
    if num_pixels_rasterized != 0:
        print(f"- '{f}'")


In [ ]:
# Creates Folder Structure from Config for years/timepoints, copying all regions
# Ensure you have filled in your config completely before running this script

config = None
with open(SNAKEFILE_CONFIG) as snakemake_config_reader:
    try:
        config = yaml.safe_load(snakemake_config_reader)
    except yaml.YAMLError as e:
        print(e)

years = config['years']
timepoints = config['timepoints']
regions_names = config['regions']

for region_name in regions_names:
    region_file_path = os.path.join(GEOJSONS_FOLDER, region_name, f'{region_name}.geojson')
    
    for year in years:
        year_folder = os.path.join(OUTPUT_DIR, str(year))
        
        for timepoint in timepoints:
            timepoint_folder = os.path.join(year_folder, str(timepoint))
            
            roi_folder = os.path.join(timepoint_folder, region_name)
            os.makedirs(roi_folder, exist_ok=True)

            shutil.copy(region_file_path, roi_folder)

In [ ]:
# Copies and adjusts APSIM file to each folder. Adjustment applied for start and end dates as defined in snakemake config for the timepoint
# Attention!: This script assumes that the APSIM file is the same for all regions and timepoints (except start/end dates)

config = None
with open(SNAKEFILE_CONFIG) as snakemake_config_reader:
    try:
        config = yaml.safe_load(snakemake_config_reader)
    except yaml.YAMLError as e:
        print(e)

years = config['years']
timepoints = config['timepoints']
regions_names = config['regions']

for year in years:
    year_folder = os.path.join(OUTPUT_DIR, str(year))

    for timepoint in timepoints:
        timepoint_folder = os.path.join(year_folder, str(timepoint))

        for roi in regions_names:  
            roi_folder = os.path.join(timepoint_folder, roi)

            start_date = config['apsim_params']['time_bounds'][year][timepoint]['sim_start_date']
            end_date = config['apsim_params']['time_bounds'][year][timepoint]['sim_end_date']
            

            with open(APSIM_TEMPLATE_PATH, "r", encoding="utf-8") as file:
                data = file.read()

                # Replace "Start" and "End" dates with new values
                data = re.sub(
                    r'"Start":\s*"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}"', 
                    f'"Start": "{start_date}T00:00:00"', 
                    data
                )
                data = re.sub(
                    r'"End":\s*"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}"', 
                    f'"End": "{end_date}T00:00:00"', 
                    data
                )

                # Write new file
                print(f'Writing new file for {roi} in {timepoint} of {year} to {roi_folder}')
               
                new_apsim_path = os.path.join(roi_folder, f'{roi}_template.apsimx')
                with open(new_apsim_path, "w", encoding="utf-8") as new_file:
                    new_file.write(data)

In [ ]:
# Now Place validation data if available in each timepoint (see docs for more info).